Sorry, this is really rough, currently this has some initial EDA I did on the various subreddits I'm thinking of scraping as well as the backbone of a potential scraping function.

Some things I need to research and/or ask about

1) I'm currently thinking about saving this as a list of nested dictionaries. Need to determine if that's the best idea or not

2) I'm worried I'm hitting reddit too much (1,000 comments per 5 subs) * (up to 1,000 posts per unique user). Researched that a bit and I think praw has a built in 2 sec delay so things are probably okay but probably should research a bit more and break up my scrapes

3) Currently, my code gives equal weight to a comment poster but that's probably not ideal, especially since people can go onto a subreddit and comment even though they're not a supporter. Would ideally like to do something where I account for this.


In [80]:
import praw
import json
import time

In [2]:
json_file = open('Reddit_creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [3]:
reddit = praw.Reddit(user_agent=json_data['user_agent'],
                     client_id=json_data['client_id'], client_secret=json_data['client_secret'])

In [4]:
yang_sub = reddit.subreddit('YangForPresidentHQ')

In [5]:
pete_sub = reddit.subreddit('Pete_Buttigieg')

In [6]:
sanders_sub = reddit.subreddit('SandersForPresident/')

In [7]:
harris_sub = reddit.subreddit('Kamala')

In [8]:
biden_sub = reddit.subreddit('JoeBiden')

In [9]:
yang_users = []
for comment in yang_sub.comments(limit=1000):
    yang_users.append(comment.author)
print(len(yang_users))
print(len(set(yang_users)))

982
505


In [16]:
setyang = set(yang_users)

In [30]:
harris_users = []
for comment in harris_sub.comments(limit=1000):
    harris_users.append(comment.author)
print(len(harris_users))
print(len(set(harris_users)))

943
215


In [31]:
setharris = set(harris_users)

In [11]:
biden_users = []
for comment in biden_sub.comments(limit=1000):
    biden_users.append(comment.author)
print(len(biden_users))
print(len(set(biden_users)))

930
197


In [19]:
setbiden = set(biden_users)

In [12]:
sanders_users = []
for comment in sanders_sub.comments(limit=1000):
    sanders_users.append(comment.author)
print(len(sanders_users))
print(len(set(sanders_users)))

986
277


In [20]:
setsanders = set(sanders_users)

In [13]:
pete_users = []
for comment in pete_sub.comments(limit=1000):
    pete_users.append(comment.author)
print(len(pete_users))
print(len(set(pete_users)))

991
171


In [21]:
setpet = set(pete_users)

In [32]:
setnotyang = set(pete_users+sanders_users+biden_users+harris_users)

In [33]:
len(setnotyang)

843

In [34]:
count = 0
for user in setyang:
    if user in setnotyang:
        print(user)
        if user in setbiden:
            print('+biden')
        if user in setharris:
            print('+harris')
        if user in setsanders:
            print('+sanders')
        if user in setpet:
            print('+pete')
        count += 1

AutoModerator
+biden
seanarturo
+sanders
hidralison_md
+harris
TheCaptain199
+biden


In [35]:
print(count)

4


Okay, feeling good, out of the 505 most recent yang posters, only 4 posted to one of these other subs, checking other subs

In [36]:

for user in setbiden:
    if user in setharris:
        print('+harris')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')


+harris
+harris
+harris
+harris
+harris
+harris
+harris
+harris
+harris
+harris
+harris
+sanders
+harris
+sanders


In [37]:

for user in setharris:
    if user in setbiden:
        print('+biden')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')

+pete
+biden
+biden
+biden
+biden
+pete
+biden
+biden
+biden
+biden
+sanders
+biden
+biden
+biden
+biden


In [43]:
count = 0
for user in setpet:
    if user in setsanders:
        count += 1
        
print(count)

0


In [45]:
len(setpet)

171

In [46]:
len(setsanders)

277

In [47]:
len(set(list(setpet)+list(setsanders)))

448

In [48]:
171+277

448

In [78]:
#backbone scraping code, idea would be to run this for every user in each subreddits set of users
yang_dict = {}
for user in yang_users[22:25]:
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if dict_sub.get(comment.subreddit.display_name,0) == 0:
            dict_sub[comment.subreddit.display_name] = 1
        else:
            dict_sub[comment.subreddit.display_name] += 1
    yang_dict[user] = dict_sub
#     print(f'finished {user}')
    time.sleep(5)

In [79]:
yang_dict

{Redditor(name='land_cg'): {'YangForPresidentHQ': 90,
  'politics': 15,
  'Instagramreality': 1,
  'Libertarian': 1,
  'The_Mueller': 1,
  'Showerthoughts': 1,
  'PublicFreakout': 2,
  'technicallythetruth': 1,
  'TheMonkeysPaw': 1,
  'nba': 4,
  'gatekeeping': 4,
  'todayilearned': 1,
  'ChapoTrapHouse': 11,
  'torontoraptors': 4,
  'TwoXChromosomes': 1,
  'tifu': 1,
  'AskReddit': 2,
  'WayOfTheBern': 2,
  'AmItheAsshole': 7,
  'watchpeoplesurvive': 3,
  'sciences': 1,
  'Bitcoin': 3,
  'JusticeServed': 1,
  'Economics': 1},
 Redditor(name='TruthWillMessYouP'): {'politics': 4,
  'YangForPresidentHQ': 16,
  'vintagejapaneseautos': 19,
  'Datsun': 62,
  'Autocross': 17,
  'projectcar': 204,
  'radiohead': 43,
  'careerguidance': 4,
  'jobs': 5,
  'personalfinance': 1,
  'FinancialCareers': 2,
  'soulwax': 3,
  'carporn': 6,
  'BusinessIntelligence': 1},
 Redditor(name='Mr_Duckerson'): {'YangForPresidentHQ': 51,
  'apple': 46,
  'MMA': 167,
  'AnimalsBeingJerks': 1,
  'news': 25,
  'Add